# Imports

In [1]:
import numpy as np
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.utils import to_categorical
from keras.layers import Dense, Activation, Conv2D,Conv2DTranspose, Dropout, Reshape, MaxPooling2D, Flatten
from keras.models import Sequential, load_model
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam


from sklearn.model_selection import train_test_split

2023-08-08 09:26:03.884332: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-08 09:26:04.091679: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-08 09:26:04.094211: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 09:26:08.884006: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Data importing and pre-processing

In [2]:
df = pd.read_csv('dataset.csv', header=None)


df = df.values.reshape(60, 64, 64, 1)

labels = np.zeros(60)

x_real_train, x_real_test = train_test_split(df, test_size=0.2) #12 test values
y_real_train, y_real_test = train_test_split(labels, test_size=0.2)

# Generator Model

Create points in latent space to be fed into generator

In [3]:
def generate_generator_input():
    input = np.random.normal(3,2.5,size=(1,100))
    
    return input

In [4]:
def make_generator():
    model = Sequential()
    
    model.add(Dense(60*8*8, input_shape=(100,)))
    model.add(Activation("relu"))
    model.add(Reshape((8,8,60)))
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(8,8,60)))
    model.add(Activation("relu"))
  
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(16,16,60)))
    model.add(Activation("relu"))

    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(32,32,60)))
    model.add(Activation("tanh"))
    
              
    model.summary()
    
    return model
    
    

# Discriminator Model

In [73]:

def make_discriminator():
    
    # 1st set of layers
    model = Sequential()
    model.add(Conv2D(128, (5,5), strides=(2,2), padding="same", input_shape=(64,64,1)))
    model.add(Activation("relu"))
    model.add(Dropout(0.3))

    model.add(Conv2D(256, (5,5), strides=(2,2), padding='same'))
    model.add(Activation("relu"))
    model.add(Dropout(0.3))
    #model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
  
    
    # output layer
    model.add(Flatten())
    model.add(Flatten())
    
    model.add(Dense(1)) # Binary classification (2 outputs), so only 1 dense layer needed
    
    model.summary()
    return model
    

# Training

First, create the models from the functions

In [63]:
gen_model = make_generator()

noise = generate_generator_input()
generated_map = gen_model(noise, training = False)

print(generated_map)



Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 3840)              387840    
                                                                 
 activation_28 (Activation)  (None, 3840)              0         
                                                                 
 reshape_6 (Reshape)         (None, 8, 8, 60)          0         
                                                                 
 conv2d_transpose_18 (Conv2  (None, 16, 16, 1)         61        
 DTranspose)                                                     
                                                                 
 activation_29 (Activation)  (None, 16, 16, 1)         0         
                                                                 
 conv2d_transpose_19 (Conv2  (None, 32, 32, 1)         2         
 DTranspose)                                         

Next, test the untrained discriminator on the map of noise generated before
Negative values means fake, positive means real

In [74]:
disc_model = make_discriminator()
decision = disc_model(generated_map)
print(decision)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 32, 128)       3328      
                                                                 
 activation_34 (Activation)  (None, 32, 32, 128)       0         
                                                                 
 dropout_3 (Dropout)         (None, 32, 32, 128)       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 256)       819456    
                                                                 
 activation_35 (Activation)  (None, 16, 16, 256)       0         
                                                                 
 dropout_4 (Dropout)         (None, 16, 16, 256)       0         
                                                                 
 flatten_5 (Flatten)         (None, 65536)           

## Loss and Optimizers

In [75]:
cross_entropy = BinaryCrossentropy(from_logits=True)

Discriminator loss, adapted from: https://www.tensorflow.org/tutorials/generative/dcgan

In [76]:
def discrim_loss(real_output, fake_output):
    real_loss = cross_entropy(np.ones_like(real_output), real_output)
    fake_loss = cross_entropy(np.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

Generator loss

In [77]:
def generator_loss(fake_output):
    return cross_entropy(np.ones_like(fake_output), fake_output)

Optimizers:

In [78]:
gen_optimizer = Adam(1e-4)
disc_optimizer = Adam(1e-4)

## Training

training parameters:

In [79]:
N_EPOCHS = 50

VERBOSE = 1


Training step

In [83]:
@tf.function
def training_step(current_batch):
    
    noise_sample = generate_generator_input()

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

        generated_map = gen_model(noise_sample, training=True)

        real_output = disc_model(current_batch, training=True)
        fake_output = disc_model(generated_map,  training = True)

        gen_loss = generator_loss(fake_output=fake_output)
        disc_loss = discrim_loss(real_output=real_output, fake_output=fake_output)

        

    gen_gradients = gen_tape.gradient(gen_loss, gen_model.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, disc_model.trainable_variables)

    gen_optimizer.apply_gradients(zip(gen_gradients, gen_model.trainable_variables))
    disc_optimizer.apply_gradients(zip(disc_gradients, disc_model.trainable_variables))

    return gen_loss, disc_loss

Train models

In [84]:
def train(dataset, N_EPOCHS):

    gen_losses = []
    disc_losses = []
    

    for epoch in range(N_EPOCHS):

        for map_batch in dataset:
            print(map_batch.shape)
            gen_loss, disc_loss = training_step(map_batch)

            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)
    
    input_for_map_after_training = generate_generator_input()
    map_after_training = gen_model(input_for_map_after_training, training=False)

    print(map_after_training)

    return gen_losses, disc_losses


Train GAN

In [85]:
print(x_real_train.shape)

gen_losses, disc_losses = train(x_real_train, N_EPOCHS)

# issue with flattening in discrim, flatten is not flattening for the final dense layer. I don't know why this is the case.


(48, 64, 64, 1)
(64, 64, 1)


AttributeError: in user code:

    File "/tmp/ipykernel_104781/3602859254.py", line 10, in training_step  *
        generated_map.reshape(1,64,64)

    AttributeError: Tensor object has no attribute 'reshape'. 
            If you are looking for numpy-related methods, please run the following:
            from tensorflow.python.ops.numpy_ops import np_config
            np_config.enable_numpy_behavior()
          


graphs:

In [18]:
epoch_list = [range(0,N_EPOCHS)]
plt.plot(disc_losses, epoch_list, 'r-')
plt.plot(gen_losses, epoch_list, 'b-')
plt.title('Gen and Disc loss over epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')

NameError: name 'disc_losses' is not defined